In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [6]:
import torch
from torch import nn

from mvn.utils.minimon import MiniMon
from mvn.utils.misc import flush_cache

from mvn.ipynb import get_args, get_config, build_labels
from mvn.pipeline.setup import setup_dataloaders, setup_experiment, build_env
from mvn.pipeline.training import do_train

In [7]:
flush_cache()

is_distributed = False
master = True
device = torch.device(0)
print('using dev {}'.format(device))
    
args = get_args()

# just to get a feeling of the dataset
# labels, mask, indices = build_labels(config.dataset.train.labels_path, 10000)
# labels, mask, indices = build_labels(config.dataset.train.labels_path, 10, allowed_subjects=['S9', 'S11'])

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  258914 KB |  607517 KB |    3357 MB |    3104 MB |
|       from large pool |  250752 KB |  589568 KB |    3285 MB |    3040 MB |
|       from small pool |    8162 KB |   18349 KB |      71 MB |      63 MB |
|---------------------------------------------------------------------------|
| Active memory         |  258914 KB |  607517 KB |    3357 MB |    3104 MB |
|       from large pool |  250752 KB |  589568 KB |    3285 MB |

In [8]:
config = get_config(args)
do_train(None, None, config, device, is_distributed, master)

cam2cam model (canonpose):
                    R_backbone has    6374406 params (~ 50.0) %
                    t_backbone has    6371331 params (~ 50.0) %
total params:             12745737
cam2cam estimation => adding 12745737 params to grad ...
  training dataset length: 12
  validation dataset length: 5
Experiment name: human36m_alg_AlgebraicTriangulationNet@13.05.2021-16:18:01
#[do_train]: epoch    0 has started!
#[cam2cam]: training batch iter 0 avg per sample loss: GEO ~ 1.927, TRANS ~ 1264.110, POSE ~ 25490.938, ROTO ~ 2.875, 3D ~ 1129.945, TOTAL ~ 1129.945
#[cam2cam]: training batch iter 1 avg per sample loss: GEO ~ 1.882, TRANS ~ 1170.481, POSE ~ 33245.715, ROTO ~ 2.896, 3D ~ 1166.999, TOTAL ~ 1166.999
#[epoch]: training MPJPE relative to pelvis: 6272.962 mm
#[epoch]: eval MPJPE relative to pelvis: 5609.638 mm
#[do_train]: epoch time ~ 3.3" => 1085 epochs / hour, 26039 epochs / day
#[do_train]: epoch    0 complete!
     BB forward pass x          3 ~        0.0 [min:        0.